In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [3]:
import analyze_utils

checkpoint_folder = "/home/jxm3/research/retrieval/inversion/saves/98b1418d38c3f9333b17ab20bff06ff9/"
trainer = analyze_utils.load_trainer(checkpoint_folder)

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wre
================ End trainer sanity check ================


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [29]:
import torch

model = trainer.inversion_trainer.model
model.use_frozen_embeddings_as_input = True

def embed_text(t: str) -> torch.Tensor:
    tok = trainer.embedder_tokenizer(t, return_tensors='pt', max_length=32)
    tok = tok.to(trainer.args.device)
    with torch.no_grad():
        output = model.call_embedding_model(**tok)
    return output

def text_from_embedding(e: torch.Tensor) -> str:
    fake_embedder_input_ids = torch.ones([1], dtype=torch.long, device=trainer.args.device)
    fake_embedder_attention_mask = torch.ones([1], dtype=torch.long, device=trainer.args.device)
    gen_tokens = model.generate(
        inputs={
            'embedder_input_ids': fake_embedder_input_ids,
            'embedder_attention_mask': fake_embedder_attention_mask,
            'frozen_embeddings': e,
        },
        generation_kwargs={
            "max_length": 32,
            "early_stopping": True,
            "num_beams": 1,
            "do_sample": False,
        },
    )
    return trainer.embedder_tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

In [30]:
model.embedder_tokenizer("Luke Skywalker is a fictional character and the protagonist of the original film trilogy of the Star Wars franchise created by George Lucas.")

{'input_ids': [12020, 5643, 24063, 19, 3, 9, 24460, 1848, 11, 8, 21326, 13, 8, 926, 814, 29093, 13, 8, 2042, 10265, 7884, 990, 57, 3080, 18831, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
len([12020, 5643, 24063, 19, 3, 9, 24460, 1848, 11, 8, 21326, 13, 8, 926, 814, 29093, 13, 8, 2042, 10265, 7884, 990, 57, 3080, 18831, 5, 1])

27

In [34]:
s = "Luke Skywalker is a fictional character and the protagonist of the original film trilogy of the Star Wars franchise created by George Lucas. Portrayed by Mark Hamill, Luke first appeared in Star Wars (1977)"

In [35]:
text_from_embedding(embed_text(s))

'Luke Skywalker Luke Skywalker is a fictional character in the "Star Wars" franchise of films created by George Lucas and the protagonist of the trilogy.'

In [37]:
h = s
print("initial:", h)
for step in range(10):
    h = text_from_embedding(embed_text(h))
    print(f"step {step}: {h}")

initial: Luke Skywalker is a fictional character and the protagonist of the original film trilogy of the Star Wars franchise created by George Lucas. Portrayed by Mark Hamill, Luke first appeared in Star Wars (1977)
step 0: Luke Skywalker Luke Skywalker is a fictional character in the "Star Wars" franchise of films created by George Lucas and the protagonist of the trilogy.
step 1: Luke Skywalker Luke Skywalker is a fictional character in the "Star Wars" franchise of films created by George Lucas and the protagonist of the trilogy.
step 2: Luke Skywalker Luke Skywalker is a fictional character in the "Star Wars" franchise of films created by George Lucas and the protagonist of the trilogy.
step 3: Luke Skywalker Luke Skywalker is a fictional character in the "Star Wars" franchise of films created by George Lucas and the protagonist of the trilogy.
step 4: Luke Skywalker Luke Skywalker is a fictional character in the "Star Wars" franchise of films created by George Lucas and the protago